In [ ]:
from bentoml.transformers import TransformersModel

In [ ]:
dc = TransformersModel.load('distilbert-base-uncased', lm_head='masked')

In [ ]:
model, tokenizer = dc.values()

In [ ]:
sentence = "A Bento box is a [MASK] box that contains food."

In [ ]:
inputs = tokenizer.encode_plus(sentence,return_tensors='pt')

In [ ]:
outputs = model(**inputs)

In [ ]:
logits = outputs.logits

In [ ]:
import torch.nn.functional as F

In [ ]:
softmax = F.softmax(logits, dim=-1)

In [ ]:
inputs

In [ ]:
masked = torch.where(inputs['input_ids'][0]==tokenizer.mask_token_id)

In [ ]:
masked, tokenizer.mask_token

In [ ]:
masked_word = softmax[0, masked, :]

In [ ]:
top_10 = torch.topk(masked_word, 10, dim = 1)[1][0]
top_10

In [ ]:
for t in top_10:
    word = tokenizer.decode([t])
    new_sentence = sentence.replace(tokenizer.mask_token, word)
    print(new_sentence)

In [ ]:
torch.set_num_threads(4)

In [1]:
from pytorch.cache_wrapped import TransformersRunnable, GPTTransformers, create_transformers_runnable

/home/aarnphm/.pyenv/versions/3.8.8/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
runnable = TransformersRunnable('sentiment_analysis', model='gpt2')

In [6]:
runnable.run("Hello")

invoking setup while calling run


loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /home/aarnphm/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_len

KeyError: "Unknown task sentiment_analysis, available tasks are ['automatic-speech-recognition', 'feature-extraction', 'text-classification', 'token-classification', 'question-answering', 'table-question-answering', 'fill-mask', 'summarization', 'translation', 'text2text-generation', 'text-generation', 'zero-shot-classification', 'conversational', 'image-classification', 'translation_XX_to_YY']"

In [7]:
from jax.experimental import stax